<a href="https://colab.research.google.com/github/egyvedriyanto78/Forecasting-Experiment/blob/main/ForecastingExp001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting Experiment #001

### Data collection was performed using the Yahoo Finance API and FRED API

Datasets:
- XAU/USD - Gold Price
- DXY - US Dollar Index
- GDP - Gross Domestic Product (GDP)
- CPIAUCSL - Inflation rate / Consumer Price Index
- FEDFUNDS - Interest Rate / Federal Funds Effective Rate
- UNRATE - Unemployment Rate

## Library

Pertama, kita perlu menginstal library yang dibutuhkan seperti pandas, requests, dan yfinance. pandas untuk pengolahan data, requests untuk menarik data dari API, dan yfinance untuk data Yahoo Finance.

In [1]:
# Install yfinance package
!pip install yfinance

In [17]:
# Install fredapi package
!pip install fredapi

In [18]:
import pandas as pd
import requests
import yfinance as yf
from datetime import datetime, timedelta
from fredapi import Fred

## Datasets

Untuk data GOLD (XAU/USD) dan DXY, kita bisa menggunakan yfinance untuk mengakses harga penutupan harian.

In [24]:
# Define start and end dates
start_date = "2013-05-03"
end_date = "2022-12-31"

# Fetch GOLD (XAU/USD) dan DXY (Dollar Index) data dari Yahoo Finance
gold_data = yf.download("GC=F", start=start_date, end=end_date)[['Close']].rename(columns={'Close': 'GOLD'})
dxy_data = yf.download("DX-Y.NYB", start=start_date, end=end_date)[['Close']].rename(columns={'Close': 'DXY'})

# Convert Yahoo Finance index to date and reset index
gold_data.index = pd.to_datetime(gold_data.index)
dxy_data.index = pd.to_datetime(dxy_data.index)

# Reset index to bring date as a column and ensure single-index
gold_data = gold_data.reset_index()
dxy_data = dxy_data.reset_index()

# Rename the index column to 'Date' to ensure consistency
gold_data.columns = ['Date', 'GOLD']
dxy_data.columns = ['Date', 'DXY']

# Display the first few rows to confirm the structure
print(gold_data.head())
print(dxy_data.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

                       Date         GOLD
0 2013-05-03 00:00:00+00:00  1464.300049
1 2013-05-06 00:00:00+00:00  1468.099976
2 2013-05-07 00:00:00+00:00  1449.000000
3 2013-05-08 00:00:00+00:00  1473.900024
4 2013-05-09 00:00:00+00:00  1468.800049
                       Date        DXY
0 2013-05-03 00:00:00+00:00  82.099998
1 2013-05-06 00:00:00+00:00  82.320000
2 2013-05-07 00:00:00+00:00  82.300003
3 2013-05-08 00:00:00+00:00  81.900002
4 2013-05-09 00:00:00+00:00  82.790001


In [25]:
# Remove timezone information from 'Date' column in gold_data and dxy_data
gold_data['Date'] = gold_data['Date'].dt.tz_localize(None)
dxy_data['Date'] = dxy_data['Date'].dt.tz_localize(None)

# Display the first few rows to confirm timezone removal
print(gold_data.head())
print(dxy_data.head())

        Date         GOLD
0 2013-05-03  1464.300049
1 2013-05-06  1468.099976
2 2013-05-07  1449.000000
3 2013-05-08  1473.900024
4 2013-05-09  1468.800049
        Date        DXY
0 2013-05-03  82.099998
1 2013-05-06  82.320000
2 2013-05-07  82.300003
3 2013-05-08  81.900002
4 2013-05-09  82.790001


Untuk data makroekonomi, kita akan menggunakan endpoint FRED API untuk GDP, CPIAUCSL, FEDFUNDS, dan UNRATE.

In [26]:
from fredapi import Fred

# Define FRED API key (ganti 'your_api_key' dengan API key Anda)
fred = Fred(api_key='c0032ef6496fbdef7acbc2ef8a82a00c')

# Fetch data dari FRED
gdp_data = fred.get_series("GDP", observation_start=start_date, observation_end=end_date)
cpi_data = fred.get_series("CPIAUCSL", observation_start=start_date, observation_end=end_date)
fedfunds_data = fred.get_series("FEDFUNDS", observation_start=start_date, observation_end=end_date)
unrate_data = fred.get_series("UNRATE", observation_start=start_date, observation_end=end_date)

# Resample ke frekuensi harian dan reset index
gdp_data_daily = gdp_data.resample('D').ffill().to_frame("GDP").reset_index()
cpi_data_daily = cpi_data.resample('D').ffill().to_frame("CPI").reset_index()
fedfunds_data_daily = fedfunds_data.resample('D').ffill().to_frame("FEDFUNDS").reset_index()
unrate_data_daily = unrate_data.resample('D').ffill().to_frame("UNRATE").reset_index()

# Display the first few rows to confirm the structure
print(gdp_data_daily.head())
print(cpi_data_daily.head())
print(fedfunds_data_daily.head())
print(unrate_data_daily.head())

       index        GDP
0 2013-04-01  16728.687
1 2013-04-02  16728.687
2 2013-04-03  16728.687
3 2013-04-04  16728.687
4 2013-04-05  16728.687
       index      CPI
0 2013-05-01  231.893
1 2013-05-02  231.893
2 2013-05-03  231.893
3 2013-05-04  231.893
4 2013-05-05  231.893
       index  FEDFUNDS
0 2013-05-01      0.11
1 2013-05-02      0.11
2 2013-05-03      0.11
3 2013-05-04      0.11
4 2013-05-05      0.11
       index  UNRATE
0 2013-05-01     7.5
1 2013-05-02     7.5
2 2013-05-03     7.5
3 2013-05-04     7.5
4 2013-05-05     7.5


In [28]:
# Rename 'index' column to 'Date' in all macroeconomic data
gdp_data_daily.rename(columns={'index': 'Date'}, inplace=True)
cpi_data_daily.rename(columns={'index': 'Date'}, inplace=True)
fedfunds_data_daily.rename(columns={'index': 'Date'}, inplace=True)
unrate_data_daily.rename(columns={'index': 'Date'}, inplace=True)

# Display the first few rows of each DataFrame to confirm the changes
print(gdp_data_daily.head())
print(cpi_data_daily.head())
print(fedfunds_data_daily.head())
print(unrate_data_daily.head())

        Date        GDP
0 2013-04-01  16728.687
1 2013-04-02  16728.687
2 2013-04-03  16728.687
3 2013-04-04  16728.687
4 2013-04-05  16728.687
        Date      CPI
0 2013-05-01  231.893
1 2013-05-02  231.893
2 2013-05-03  231.893
3 2013-05-04  231.893
4 2013-05-05  231.893
        Date  FEDFUNDS
0 2013-05-01      0.11
1 2013-05-02      0.11
2 2013-05-03      0.11
3 2013-05-04      0.11
4 2013-05-05      0.11
        Date  UNRATE
0 2013-05-01     7.5
1 2013-05-02     7.5
2 2013-05-03     7.5
3 2013-05-04     7.5
4 2013-05-05     7.5


### Merge Dataset

In [29]:
# Merge all data on 'Date' column
combined_data = gold_data.merge(dxy_data, on="Date", how='inner') \
                         .merge(gdp_data_daily, on="Date", how='inner') \
                         .merge(cpi_data_daily, on="Date", how='inner') \
                         .merge(fedfunds_data_daily, on="Date", how='inner') \
                         .merge(unrate_data_daily, on="Date", how='inner')

# Display the first few rows of the combined data for verification
print(combined_data.head())

        Date         GOLD        DXY        GDP      CPI  FEDFUNDS  UNRATE
0 2013-05-03  1464.300049  82.099998  16728.687  231.893      0.11     7.5
1 2013-05-06  1468.099976  82.320000  16728.687  231.893      0.11     7.5
2 2013-05-07  1449.000000  82.300003  16728.687  231.893      0.11     7.5
3 2013-05-08  1473.900024  81.900002  16728.687  231.893      0.11     7.5
4 2013-05-09  1468.800049  82.790001  16728.687  231.893      0.11     7.5


### Export Dataset

In [30]:
from google.colab import files
combined_data.to_csv('combined_data.csv', index=False)
files.download('combined_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>